In [1]:
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
from timeit import default_timer as timer
np.random.seed(1000)

In [13]:
def evalf(x, n, l):
   assert type(x) is np.ndarray
   return 0.5 * (np.linalg.norm(np.subtract(np.matmul(A,x) , y)))**2 + l * (np.linalg.norm(x)**2) * 0.5

In [14]:
def evalg(x, n, l):
  assert type(x) is np.ndarray
  return np.add(np.matmul(A.T, np.subtract(np.matmul(A, x), y)), l * x)

In [15]:
def evalh(x, n, l):
  assert type(x) is np.ndarray
  return np.add(np.matmul(A.T,A), l * np.identity(n))

In [16]:
def compute_B_k(s, y, B_k, n, l):
  assert type(s) is np.ndarray
  assert type(y) is np.ndarray
  assert type(B_k) is np.ndarray
  assert len(s) == n and len(y) == n
  mu = 1 / np.dot(np.squeeze(np.asarray(y)), np.squeeze(np.asarray(s)))
  I = np.identity(n)
  a1 = np.outer(np.multiply(mu, s), np.transpose(y))
  a2 = np.outer(np.multiply(mu, y), np.transpose(s))
  a = np.subtract(I, a1)
  b = np.subtract(I, a2)
  c = np.outer(np.multiply(mu, s), np.transpose(s))
  d = np.matmul(np.matmul(a, B_k), b)
  z = np.add(d, c)
  return z

In [6]:
EXACT_LINE_SEARCH = 1
BACKTRACKING_LINE_SEARCH = 2
CONSTANT_STEP_LENGTH = 3

In [17]:
def compute_D_k(x,n, l):
  assert type(x) is np.ndarray
  assert len(x) == n
  mat = evalh(x, n, l)
  return np.linalg.inv(mat)

In [18]:
def compute_steplength_backtracking_scaled_direction(x, gradf, alpha_start, rho, gamma, B_k, l, n): #add appropriate arguments to the function
  assert type(x) is np.ndarray
  assert type(gradf) is np.ndarray
  assert type(alpha_start) is float and alpha_start>=0.
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0.
  alpha = alpha_start
  p =  -gradf
  D_k = B_k

  while evalf(x + alpha*np.matmul(D_k,p), n, l) > evalf(x, n, l) + gamma * alpha* (np.matmul(np.matrix.transpose(gradf), np.matmul(D_k,p))):
    alpha = alpha * rho
  return alpha

In [19]:
def minimizer_Newtonmethod(start_x, n, tol, line_search_type , l, *args):
  assert type(start_x) is np.ndarray
  assert len(start_x) == n
  assert type(tol) is float and tol>=0
  x = start_x
  g_x = evalg(x,n, l)
  h_x = evalh(x,n, l)
  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      wrng = 'check!'
      raise ValueError(wrng)
    elif len(args)<3 :
      wrng = 'check!'
      raise ValueError(wrng)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0

  step_length = 1.
  while (np.linalg.norm(g_x) > tol):
    D_k = np.linalg.inv(evalh(x, n, l))
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x, g_x, alpha_start, rho, gamma, D_k, l, n)
    elif line_search_type == CONSTANT_STEP_LENGTH:
      step_length = 1.
    else:
      raise ValueError('check!')
    x = np.subtract(x, np.multiply(step_length,np.matmul(D_k, g_x)))
    k += 1
    g_x = evalg(x, n, l)
  return x

In [20]:
def minimizer_BFGS_scaling(start_x, n, tol, line_search_type, l, *args):
  assert type(start_x) is np.ndarray #do not allow arbitrary type arguments
  assert len(start_x) == n #do not allow arbitrary size arguments
  assert type(tol) is float and tol>=0

  x = start_x
  g_x = evalg(x,n, l)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  B_k = np.identity(n)
  x_old = x
  while (np.linalg.norm(g_x) > tol):
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x_old, g_x, alpha_start, rho, gamma, B_k, l, n)
    elif line_search_type == CONSTANT_STEP_LENGTH:
      step_length = 1.
    else:
      raise ValueError('check!')
    x_new = np.subtract(x_old, np.multiply(step_length, np.matmul(B_k,g_x)))
    g_x = evalg(x_new,n, l)
    s = np.subtract(x_new, x_old)
    y = np.subtract(evalg(x_new,n, l), evalg(x_old,n, l))
    B_k = compute_B_k(s, y, B_k, n, l)
    x_old = x_new
    k += 1
  return x_new

In [21]:
my_tol = my_tol= 1e-5
alpha = 0.9
rho = 0.5
gamma = 0.5
N = 200
ds = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 500000, 1000000]
lambda_reg = 0.001
eps = np.random.randn(N,1)

In [ ]:
#Newton method
for i in range(np.size(ds)):
  d=ds[i]
  A = np.random.randn(N,d)
  xorig = np.ones((d,1))
  my_start_x = np.zeros((d,1))
  y = np.dot(A,xorig) + eps
  start = timeit.default_timer()
  x_opt = minimizer_Newtonmethod(my_start_x, d, my_tol, BACKTRACKING_LINE_SEARCH, lambda_reg, alpha, rho, gamma)
  newtontime = timeit.default_timer() - start
  diff = np.subtract(x_opt, xorig)
  print("\n\nd:", d, "\nTotal time:", newtontime, "\nL2 norm difference ||Ax* - y||_2^2: ", np.linalg.norm(np.subtract(np.matmul(A, x_opt), y))**2,"\nL2 norm difference || x_opt - xorig||: ", np.linalg.norm(diff)**2)



d: 1000 
Total time: 1.4848241189999953 
L2 norm difference ||Ax* - y||_2^2:  2.6203554146006913e-07 
L2 norm difference || x_opt - xorig||:  793.2210356671586


${Question 1.}$

In the previous exercise, the hessian matrix was $A^TA$ but by adding reguralizer term, it is becoming $A^TA+\lambda I$. One cannot assure the invertibility of the matrix $A^TA$, so it is possible that inverse of hessian matrix doesn't exist for previous exercise but the matrix $A^TA+\lambda I$ is always invertible because $A^TA$ is symmetric and by adding positive diagonal elements to symmetric matrix, We assure the positiveness of eigenvalues.Therefore for $\lambda > 0 $, the inverse of hessian matrix always exist for regularized function in this exercise.

In [ ]:
#BFGS method
my_tol= 1e-5
alpha = 0.9
rho = 0.5
gamma = 0.5
N = 200
ds = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 500000, 1000000]
lambda_reg = 0.001
eps = np.random.randn(N,1)
for i in range(np.size(ds)):
  d=ds[i]
  A = np.random.randn(N,d)
  my_start_x = np.zeros((d,1))
  xorig = np.ones((d,1))
  y = np.dot(A,xorig) + eps

  start = timeit.default_timer()
  x_star_bfgs = minimizer_BFGS_scaling(my_start_x, d, my_tol, BACKTRACKING_LINE_SEARCH, lambda_reg, alpha, rho, gamma)

  bfgstime = timeit.default_timer() - start
  diff = np.subtract(x_star_bfgs, xorig)
  print("\n\nd:", d, "\nTotal time:", bfgstime, "\nL2 norm difference ||Ax* - y||_2^2: ", np.linalg.norm(np.subtract(np.matmul(A, x_star_bfgs), y))**2,"\nL2 norm difference || x_opt - xorig||: ", np.linalg.norm(diff)**2)

**Question 1**




**Newton's Method**

**1)**The dimension of A for the first row is (200, 1000) with a total time of 1.63,  $||Ax^* - y||_2^2$ value of $2.12 \times 10^{-7}$, and $||x^* - x{\text{orig}}||_2^2$ value of 817.72;

**2)**the dimension of A is (200, 5000) with a total time of 122.62, $||Ax^* - y||_2^2$ value of $4.74 \times 10^{-8}$, and $||x^* - x{\text{orig}}||_2^2$ value of 4774.91;

**3)**the dimension of A is (200, 10000) with a total time of 979.60,  $||Ax^* - y||_2^2$ value of $1.75 \times 10^{-8}$, and $||x^* - x{\text{orig}}||_2^2$ value of 9825.86."


**BFGS Method**

**1)**Dimension of A is (200, 1000) and the total time for the BFGS Method is 21.27 seconds. The $||Ax^* - y||_2^2$ value is approximately 0.00212, and $||x^* - x{\text{orig}}||_2^2$ is 817.72.

**2)**Dimension of A is (200, 5000) and the total time for the BFGS Method is 112.11 seconds. The $||Ax^* - y||_2^2$ value is approximately 0.000474, and $||x^* - x{\text{orig}}||_2^2$ is 4774.91.

**3)**Dimension of A is (200, 10000) and the total time for the BFGS Method is 906.31 seconds. The $||Ax^* - y||_2^2$ value is approximately 0.000175, and $||x^* - x{\text{orig}}||_2^2$ is 9825.86.

**Question 2**

The failure occures for newton's method for size of A = (200, 10000)

The failure occures for BFGS method for size of A = (200, 10000)